In [1]:
%reload_ext autoreload
%autoreload 2
from EnzymePynetics import Estimator

In [2]:
estimator, enzymeml = Estimator.from_enzymeml(
    enzymeml="simulated_enzymeML.json",
    measured_reactant="substrate",
)

In [3]:
substrate, product = estimator.reactants
enzyme = estimator.enzymes[0]

In [4]:
# Define Reaction
oxidation = estimator.add_reaction(
    id="r1", name="Oxidation", educt=substrate, product=product, enzyme=enzyme
)

In [5]:
# Define potential substrate rate-laws
michaelis = estimator.add_model(
    id="model1",
    name="michaelis-menten",
    equation="substrate = -substrate * enzyme * k_cat / (K_m + substrate)",
)

competitive_product = estimator.add_model(
    id="model3",
    name="competitive product inhibition",
    equation="substrate = -substrate * enzyme * k_cat / (K_m * (1 + product / K_ic) + substrate)",
)

substrate_inhibition = estimator.add_model(
    id="model4",
    name="substrate inhibition",
    equation="substrate = -k_cat * enzyme * substrate / (K_m + ((1+(substrate/K_iu))*substrate))",
)

# Define potential enzyme rate-laws
enzyme_inactivation = estimator.add_model(
    id="model2",
    name="enzyme inactivation",
    equation="enzyme = -k_ie * enzyme",
)

In [6]:
estimator.fit_models()

/Users/max/miniconda3/envs/pynetics/lib/python3.11/site-packages/sdRDM/base/datamodel.py:326: UserWarning: No 'URL' and 'Commit' specified. This model might not be re-usable.
  warnings.warn(


Reactant
├── id
├── name
├── vessel_id
├── init_conc
├── constant
├── unit
├── uri
├── creator_id
├── smiles
├── inchi
├── chebi_id
└── ontology


TypeError: Can't compare 'Reactant' to type ''

In [ ]:
estimator.visualize()

: 

In [ ]:
# Get the best model system
model_system = estimator.get_reaction_system(
    "competitive product inhibition with enzyme inactivation"
)

# Safe the model system to the EnzymeML document
enzymeml_with_kinetic_model = estimator.to_enzymeml(enzymeml, model_system)

: 

In [ ]:
for reaction in enzymeml_with_kinetic_model.reactions:
    print(reaction)

: 